<h1 style="color:steelblue; font-family:Ewert; font-size:200%;" class="font-effect-3d">Code Library, Style and Links</h1>

The previous notebooks:

- [Quick, Draw! Doodle Recognition 1](https://www.kaggle.com/olgabelitskaya/quick-draw-doodle-recognition-1)

- [Quick, Draw! Doodle Recognition 2](https://www.kaggle.com/olgabelitskaya/quick-draw-doodle-recognition-2)

- [Quick, Draw! Doodle Recognition 1B](https://www.kaggle.com/olgabelitskaya/quick-draw-doodle-recognition-1b)

- [Quick, Draw! Doodle Recognition 2B](https://www.kaggle.com/olgabelitskaya/quick-draw-doodle-recognition-2b)

- [Quick, Draw! Doodle Recognition 1CV](https://www.kaggle.com/olgabelitskaya/quick-draw-doodle-recognition-1cv)

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d|ice|');
span {font-family:'Roboto'; color:black; text-shadow: 5px 5px 5px #aaa;}  
div.output_area pre{font-family:'Roboto'; font-size:110%; color: steelblue;}      
</style>

In [ ]:
import numpy as np 
import pandas as pd 
import keras as ks

import os
import ast
import cv2
import warnings

import matplotlib.pylab as plt
%matplotlib inline

os.listdir("../input")

In [ ]:
# list of labels
files = os.listdir("../input/quickdraw-doodle-recognition/train_simplified")
file_path = '../input/quickdraw-doodle-recognition/train_simplified/'
labels = [el.replace(" ", "_")[:-4] for el in files]
print(sorted(labels)) # 340 labels - 17 sets

In [ ]:
# files with weights for 34 label sets
weights = sorted(os.listdir('../input/quick-draw-model-weights-for-doodle-recognition/weights_cv/'))
weight_path = '../input/quick-draw-model-weights-for-doodle-recognition/weights_cv/'
print(weights)

In [ ]:
I = 64 # image size in pixels
T = 20 # number of labels in one set

In [ ]:
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation, Dropout, Dense
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

warnings.filterwarnings('ignore', category=UserWarning)
plt.style.use('seaborn-whitegrid')
style_dict = {'background-color':'gainsboro', 
              'color':'steelblue', 
              'border-color': 'white', 
              'font-family':'Roboto'}

In [ ]:
# additional functions
def display_drawing():
    plt.figure(figsize=(16,30))
    plt.suptitle('Test Pictures')
    for i in range(20):
        picture = ast.literal_eval(test_data.drawing.values[i])
        for x,y in picture:
            plt.subplot(10,2,i+1)
            plt.plot(x, y, '-o', color='gainsboro')
            plt.xticks([]); plt.yticks([])
            plt.title(presubmission.iloc[i][1])
        plt.gca().invert_yaxis()
        plt.axis('equal');
            
def get_image(data, lw=6, time_color=True):
    data = ast.literal_eval(data)
    image = np.zeros((280, 280), np.uint8)
    for t, s in enumerate(data):
        for i in range(len(s[0]) - 1):
            color = 255 - min(t, 10) * 15 if time_color else 255
            _ = cv2.line(image, (s[0][i]+10, s[1][i]+10),(s[0][i+1]+10, s[1][i+1]+10), color, lw) 
    return cv2.resize(image, (I, I))

def test_predict(data):
    images = []
    images.extend([get_image(data.drawing.iloc[i]) for i in range(len(data))])    
    images = np.array(images)
    
    model.load_weights(weight_path + weights[0])
    predictions = model.predict(images.reshape(-1,I,I,1))
    
    for w in weights[1:]:
        w = weight_path + w
        model.load_weights(w)
        predictions2 = model.predict(images.reshape(-1,I,I,1))
        predictions = np.concatenate((predictions, predictions2), axis=1)
        
    return predictions

<h1 style="color:steelblue; font-family:Ewert; font-size:200%;" class="font-effect-3d">The Model</h1>

In [ ]:
# the model architecture
def model():
    model = Sequential()
    
    model.add(Conv2D(32, (5, 5), padding='same', input_shape=(I,I,1)))
    model.add(LeakyReLU(alpha=0.02))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(196, (5, 5)))
    model.add(LeakyReLU(alpha=0.02))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(GlobalMaxPooling2D())
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.5)) 
    
    model.add(Dense(T))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = model()

<h1 style="color:steelblue; font-family:Ewert; font-size:200%;" class="font-effect-3d">Test Predictions</h1>

In [ ]:
# read the test file
test_data = pd.read_csv('../input/quickdraw-doodle-recognition/test_simplified.csv',
                        index_col='key_id')
test_data.tail(3).T.style.set_properties(**style_dict)

In [ ]:
# predictions for each label set
test_predictions = test_predict(test_data)

In [ ]:
# 10 best guesses among all label sets
test_labels = [[ labels[i] for i in test_predictions[k].argsort()[-10:][::-1] ] \
               for k in range(len(test_predictions))]
test_labels = [ " ".join(test_labels[i]) for i in range(len(test_labels))]

presubmission = pd.DataFrame({"key_id": test_data.index, "word": test_labels})
presubmission.to_csv('submission_10best.csv', index=False)

In [ ]:
display_drawing()

In [ ]:
# 3 best guesses among all label sets
test_labels = [[ labels[i] for i in test_predictions[k].argsort()[-3:][::-1] ] \
               for k in range(len(test_predictions))]
test_labels = [ " ".join(test_labels[i]) for i in range(len(test_labels))]

submission = pd.DataFrame({"key_id": test_data.index, "word": test_labels})
submission.to_csv('submission.csv', index=False)

<h1 style="color:steelblue; font-family:Ewert; font-size:200%;" class="font-effect-3d">In Progress</h1>